# Imports and Requirements
For this project, we need the data sets and some python libraries

In [1]:
import pandas as pd
import numpy as np

## Reading and cleaning the data

In [3]:
df_original = pd.read_csv('./data/chemicals-in-cosmetics.csv')
df = df_original.drop_duplicates()
print('The original data had ', df_original.shape[0], "rows")
print('After removing duplicates, the data has', df.shape[0], "rows")

The original data had  112870 rows
After removing duplicates, the data has  112616 rows
